In [1]:
!wget https://cernbox.cern.ch/remote.php/dav/public-files/ZUHveJKajnZNwTA/QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet
!wget https://cernbox.cern.ch/remote.php/dav/public-files/ZUHveJKajnZNwTA/QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540.test.snappy.parquet
!wget https://cernbox.cern.ch/remote.php/dav/public-files/ZUHveJKajnZNwTA/QCDToGGQQ_IMGjet_RH1all_jet0_run2_n55494.test.snappy.parquet

--2024-03-28 12:01:18--  https://cernbox.cern.ch/remote.php/dav/public-files/ZUHveJKajnZNwTA/QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet
Resolving cernbox.cern.ch (cernbox.cern.ch)... 137.138.120.151, 128.142.170.17, 128.142.53.28, ...
Connecting to cernbox.cern.ch (cernbox.cern.ch)|137.138.120.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184115523 (176M) [application/octet-stream]
Saving to: 'QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet'

QCDToGGQQ_IMGjet_RH 100%[===================>] 175.59M  14.8MB/s    in 15s     

2024-03-28 12:01:39 (11.9 MB/s) - 'QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet' saved [184115523/184115523]

--2024-03-28 12:01:40--  https://cernbox.cern.ch/remote.php/dav/public-files/ZUHveJKajnZNwTA/QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540.test.snappy.parquet
Resolving cernbox.cern.ch (cernbox.cern.ch)... 137.138.120.151, 128.142.170.17, 128.142.53.28, ...
Connecting to cernbox.cern.ch 

In [2]:
import time
import numpy as np
import cv2
import pyarrow.parquet as pq
import os

start_time = time.time()

def generate(pf, path, ab):
    record_batch = pf.iter_batches(batch_size=4*1024)
    while True:
        try:
            batch = next(record_batch)
            ab = transform(batch, path, ab)
        except StopIteration:
            return ab

def transform(batch, path, ab):
    p = batch.to_pandas()
    im = np.array(np.array(np.array(p.iloc[:, 0].tolist()).tolist()).tolist())
    meta = np.array(p.iloc[:, 3])
    return saver(im, meta, path, ab)

def saver(im, meta, path, ab):
    alpha, beta = ab
    im[im < 1.e-3] = 0  
    for i in range(3): 
        im[:, i, :, :] = (im[:, i, :, :] - im[:, i, :, :].mean()) / (im[:, i, :, :].std())
        im[:, i, :, :] = np.clip(im[:, i, :, :], 0, 500 * im[:, i, :, :].std())
        im[:, i, :, :] = (im[:, i, :, :] / im[:, i, :, :].max())
    for i in range(meta.shape[0]):
        img = im[i].astype(np.uint8).transpose(1, 2, 0)
        if meta[i] == 0:
            impath = os.path.join(path, "0", f"{alpha}.png")
            alpha += 1
        elif meta[i] == 1:
            impath = os.path.join(path, "1", f"{beta}.png")
            beta += 1
        cv2.imwrite(impath, img)
    return [alpha, beta]

def runner(source_files, target_directory):
    ab = [0, 0]
    for file in source_files:
        parquet_file_path = os.path.join('/kaggle/working/', file)
        ab = generate(pq.ParquetFile(parquet_file_path), target_directory, ab)
    print("Data generation completed")

input_files = [
    'QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet',
    'QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540.test.snappy.parquet',
    'QCDToGGQQ_IMGjet_RH1all_jet0_run2_n55494.test.snappy.parquet'
]

output_directory = '/kaggle/working/output/'

# Create output directories
for class_label in ['0', '1']:
    os.makedirs(os.path.join(output_directory, class_label), exist_ok=True)

runner(source_files=input_files, target_directory=output_directory)

end_time = time.time()
Running_time = end_time - start_time
print(f"Time: {Running_time} seconds")

Data generation completed
Time: 605.7412676811218 seconds
